In [ ]:
import requests
from typing import Dict, Union, Optional

NAVER_API_URL: str = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode'
NAVER_API_HEADERS: Dict[str, str] = {
    'X-NCP-APIGW-API-KEY-ID': 'Private_ID',
    'X-NCP-APIGW-API-KEY': 'Private_KEY'
}

def request_naver_api(address: str) -> Dict:
    try:
        response = requests.get(
            NAVER_API_URL,
            params={'query': address},
            headers=NAVER_API_HEADERS
        )
        data: Dict = response.json()
        return data
    except Exception as error:
        raise error

def parse_address_data(data: Dict) -> Union[Dict[str, str], bool]:
    if data['meta']['totalCount'] == 0:
        return False

    Longitude: str = data['addresses'][0]['x']
    Latitude: str = data['addresses'][0]['y']

    return {'Longitude': Longitude, 'Latitude': Latitude}

def get_address_details(add1: str, add2: str) -> Dict[str, str]:
    data: Dict = request_naver_api(add1)
    result1: Union[Dict[str, str], bool] = parse_address_data(data)

    if not result1:
        data = request_naver_api(add2)
        result2: Union[Dict[str, str], bool] = parse_address_data(data)

        if not result2:
            return {'x': "", 'y': ""}

        return result2

    return result1

import pandas as pd

df = pd.read_csv('/content/seoul_apart_price.csv')
#encoding='cp949
'
df['address'] = df.apply(lambda row: get_address_details(row['지번주소'], row['도로명주소']), axis=1)

#df을 딕셔너리로 변환
rows_list = df.to_dict('records')

merged_list = []

for row in rows_list:

    address = row.pop('address')
    merged = {**row, **address}
    merged_list.append(merged)

new_df = pd.DataFrame(merged_list)

# 새 DataFrame을 CSV 파일로 저장
new_df.to_csv('/content/seoul_apart_price.csv', index=False)

<ipython-input-2-4cbb234a9502>:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unique_address_df['address_details'] = unique_address_df.apply(lambda row: get_address_details(row['Address'], row['도로명주소']), axis=1)


  0%|          | 0/16557 [00:00<?, ?it/s]

총 소요 시간: 8722.327285051346 초
